In [1]:
import numpy as np
from main_6sept import LeapNode_Poscontrol, LeapNode_Taucontrol
import time
import mujoco

In [2]:
leap_hand = LeapNode_Poscontrol()
pos=leap_hand.read_pos_leap()
print(pos)

ERROR:root:> write_byte: [Motor ID: 0] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 1] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 2] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 3] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 4] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 5] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 6] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 7] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 8] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 9] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 10] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 11] [TxRxResult] There is no status packet!
ERROR:root:> write_byte: [Motor ID: 12] [TxRxResul

[-0.05976666  1.4251269  -0.05823268  0.56456362  0.12431108  1.49262177
  0.18567024  0.19334017  0.15345679  1.35916553 -0.19015492  0.09363161
  1.52483545  0.01846634 -0.16100921  0.22862183]


In [3]:
def J(model,data,site_name):
        # model=mujoco.MjModel.from_xml_path(xml_path)
        # data = mujoco.MjData(model)
        mujoco.mj_forward(model, data)
        jacp = np.zeros((3, model.nv))  # translation jacobian
        jacr = np.zeros((3, model.nv)) 

        site_id=model.site(site_name).id
        mujoco.mj_jacSite(model, data, jacp, jacr, site_id)

        return np.vstack((jacp, jacr))

Kp = 1.0  # Proportional gain
Kd = 0.1  # Derivative gain
# Initialize Kp, Kd, and prev_pos
Kp = 1.0  # Proportional gain (set according to your control design)
Kd = 0.1  # Derivative gain (set according to your control design)
prev_pos = leap_hand.read_pos_leap()  # Initialize prev_pos to the current positions
dt = 0.01  # Default dt (for the first iteration)
t1 = time.time()



In [4]:
pos_index= pos[0:4].reshape(4)
pos_index_mujoco=[pos_index[1], pos_index[0], pos_index[2], pos_index[3]]

pos_thumb=pos_thumb_mujoco= pos[12:16].reshape(4)


index_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)


index_d.qpos= pos_index_mujoco
mujoco.mj_forward(index_m, index_d)
index_J=J(index_m,index_d,'contact_index')
print(index_J)

thumb_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=pos_thumb_mujoco
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J=J(thumb_m,thumb_d,'contact_thumb')
print(thumb_J)

[[-1.14340345e-01  5.12701772e-04 -6.17587072e-02 -2.64060135e-02]
 [ 0.00000000e+00 -7.40953646e-02 -1.32384332e-03 -1.44933880e-03]
 [ 6.43448585e-03 -3.49469473e-03  1.33001045e-02  2.06063550e-02]
 [ 0.00000000e+00 -9.89408956e-01 -8.67025450e-03 -8.67025450e-03]
 [ 1.00000000e+00  0.00000000e+00  9.98214504e-01  9.98214504e-01]
 [ 0.00000000e+00 -1.45154803e-01  5.90984747e-02  5.90984747e-02]]
[[ 1.18016273e-18 -4.06113777e-04  9.31960984e-02  4.72066654e-02]
 [-1.24399163e-01 -5.73684752e-03 -1.98352470e-03 -1.47830549e-03]
 [-5.31498042e-03 -2.63856341e-04  5.66428110e-03  1.31660682e-02]
 [-1.00000000e+00  0.00000000e+00 -1.84652896e-02 -1.84652896e-02]
 [ 0.00000000e+00  4.59446942e-02 -9.98773667e-01 -9.98773667e-01]
 [-2.22044605e-16 -9.98943985e-01 -4.59368607e-02 -4.59368607e-02]]


In [5]:
F_index = np.reshape([-0.30, 0, 0, 0, 0, 0], [6, 1])
F_thumb=np.reshape([0.30, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index = index_J.T @ F_index
Tau_index[[0, 1]] = Tau_index[[1, 0]]

Tau_thumb = thumb_J.T @ F_thumb

# Convert torque values to float
Tau_index = [float(torque[0]) for torque in Tau_index]
Tau_thumb = [float(torque[0]) for torque in Tau_thumb]

In [6]:
leap_pos = LeapNode_Poscontrol()

q0 = np.zeros(16) # 16-element array
v0 = np.zeros(16)  # initial velocity (array of zeros)
q1 = pos.reshape(16)  # final positions
v1 = np.zeros(16)  # final velocity (array of zeros)
t0 = 3 # initial time
t1 = 7

# q2=np.array([a1,b1,c1,d1, 0,0,0,0,0,0,0,0,e1,f1,g1,h1])
# v2=np.zeros(16)

start_time = time.time()

while time.time() - start_time < t0:
    leap_pos.set_allegro(np.zeros(16))
    time.sleep(0.03)

while time.time() - start_time < t1 and time.time()>t0:
    current_time=time.time()
    # Ensure the current time falls within [t0, t1]
    
    qd = leap_pos.cubic_trajectory(q0, v0, q1, v1, t0, t1, current_time)
    leap_pos.set_allegro(qd)
    # time.sleep(0.03)

leap_torque=LeapNode_Taucontrol()
# while True:
#     leap_pos.set_allegro(qd)

# Apply torque
# while time.time() - start_time < 15 and time.time() > t1:

#     curr_pos=leap_hand.read_pos_leap()
#     # Tau_thumb[0]=Kp*(pos[12]-curr_pos[12])
#     prev_pos = np.zeros_like(curr_pos)  # Initialize to the same shape as curr_pos

#     # Compute velocity (derivative term)
#     velocity = (curr_pos[12] - prev_pos[12]) /

#     # Update control law with Kp and Kd terms
#     Tau_thumb[0] = Kp * (pos[12] - curr_pos[12]) + Kd * (-velocity)

#     # After updating Tau_thumb, store current position for the next time step
#     prev_pos[12] = curr_pos[12]
# #while True:
#     leap_torque.set_desired_torque(Tau_index+ [0,-0.1,0,0,0,-0.1,0,0]+Tau_thumb)
    
#     print("Torque given:", Tau_index+ [0,-0.1,0,0,0,-0.1,0,0]+Tau_thumb)

start_time = time.time()
while time.time() - start_time < 15 and time.time() > t1:
    
    # Get the current position of the hand
    curr_pos = leap_hand.read_pos_leap()
    
    # Compute the velocity (derivative term)
    velocity = (curr_pos[12] - prev_pos[12]) / dt
    
    # Update control law with Kp and Kd terms
    Tau_thumb[0] = Kp * (pos[12] - curr_pos[12]) + Kd * (-velocity)
    
    # Update desired torques (this assumes Tau_index is defined elsewhere)
    desired_torque = Tau_index + [0, -0.1, 0, 0, 0, -0.1, 0, 0] + Tau_thumb
    
    # Set the desired torque to the system
    leap_torque.set_desired_torque(desired_torque)
    
    # Print the applied torque
    print("Torque given:", desired_torque)
    
    # Update prev_pos and prev_time for the next iteration
    prev_pos[12] = curr_pos[12]
    prev_time = time.time()
    
    # Calculate dt for the next iteration (real-time step)
    dt = time.time() - prev_time
    
    # Sleep if necessary to maintain loop frequency
    time.sleep(max(0, 0.01 - dt))  # Adjust based on desired frequency


desired_torque: [-0.00015381053158972734, 0.03430210340648505, 0.018527612173446714, 0.007921804052876564, 0, -0.1, 0, 0, 0, -0.1, 0, 0, np.float64(0.2193598747253418), -0.0001218341331253897, 0.027958829512440275, 0.014161999626287195]
desired_current: [-3.01589278e-04  6.72590263e-02  3.63286513e-02  1.55329491e-02
  0.00000000e+00 -1.96078431e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.96078431e-01  0.00000000e+00  0.00000000e+00
  4.30117401e-01 -2.38890457e-04  5.48212343e-02  2.77686267e-02]
Torque given: [-0.00015381053158972734, 0.03430210340648505, 0.018527612173446714, 0.007921804052876564, 0, -0.1, 0, 0, 0, -0.1, 0, 0, np.float64(0.2193598747253418), -0.0001218341331253897, 0.027958829512440275, 0.014161999626287195]
desired_torque: [-0.00015381053158972734, 0.03430210340648505, 0.018527612173446714, 0.007921804052876564, 0, -0.1, 0, 0, 0, -0.1, 0, 0, np.float64(-12868.010737895966), -0.0001218341331253897, 0.027958829512440275, 0.014161999626287195]
desired_curr

OverflowError: can't convert negative int to unsigned

In [ ]:
# Scaling factor for gradual reduction
scale_factor = 0.999  # Reduce by 1% in each loop iteration



while 15 <= (time.time() - start_time) <= 18:
    print("retract1")
    # Gradually reduce the forces
    F_index = scale_factor * F_index
    F_thumb = scale_factor * F_thumb

    # Recompute the torque values
    Tau_index = index_J.T @ F_index
    Tau_index[[0, 1]] = Tau_index[[1, 0]]

    Tau_thumb = thumb_J.T @ F_thumb

    # Convert torque values to float
    Tau_index = [float(torque[0]) for torque in Tau_index]
    Tau_thumb = [float(torque[0]) for torque in Tau_thumb]

    # Apply the updated torque
    leap_torque.set_desired_torque(Tau_index + [0, -0.1, 0, 0, 0, -0.1, 0, 0] + Tau_thumb)

leap_pos=LeapNode_Poscontrol()

while 18 <= (time.time() - start_time) <= 23:
    
    leap_pos.set_allegro([0,np.pi/2,0,0,0,0,0,0,0,0,0,0,np.pi/2,0,0,0])